In [1]:
import numpy as np
import pandas as pd
import ProGED as pg
from DN4_2_podatky import *

In [85]:
from nltk import Nonterminal

np.random.seed(1234)

gramatika = "E -> E '+' F [0.8]| E '-' F [0.1]| F [0.1] \n"
gramatika += "F -> F '*' T [0.2]| F '/' T [0.4]| T [0.4] \n"
gramatika += "T -> V [0.4]| '('E')' [0.3]| 'sin' '('E')'[0.3] \n"
gramatika += "V -> 'x1' [0.2]| 'x2' [0.2]| 'x3' [0.2] | 'x4' [0.2] | 'x5' [0.2]"


grammar = pg.GeneratorGrammar(gramatika)
podatki = generiraj_enacbo_1(100)
ED = pg.EqDisco(data=podatki, 
                sample_size=100,
                lhs_vars=["y"],
                rhs_vars=["x1", "x2", "x3", "x4", "x5"],
                verbosity=1,
                generator = grammar)
#pg.Model
ED.generate_models()
mm = ED.fit_models()
rm = ED.get_results()



[x2*x4]
[x4]
[x2 + x3]
[x3/x1]
[x3/x1]
[x3/x1]
[x1]
[-x1**2 + x4/x3]
[1/x4]
[x3 + x4]
[x3 + x4]
[x1 + x3]
[x1 + x2/x5]
[x1 + x2/x5]
[-x2 + x3/x2]
[sin(x5)]
[x3]
[x5]
[x2/x5]
[x2 + x4]
[x3 + x5]
[x1 + x2]
[x4 + x5]
[x4 + x5]
[x4 + x5]
[x4 + x5]
[x4 + x5]
[x4 + x5]
[x4**2]
[x3/(x4*x5) + x4]
[x3/(x4*x5) + x4]
[x3/(x4*x5) + x4]
[x3/(x4*x5) + x4]
[x2]
[x2]
[x2]
[x2/x4 + x3**2 + x5/x2]
[x2/x4 + x3**2 + x5/x2]
[x2/x4 + x3**2 + x5/x2]
[-x4*x5/sin(x3 - x4)]
[-x4*x5/sin(x3 - x4)]
[x1/x3]
[x1/x3]
[x1/x3]
[x1/x3]
[x1 + 2*x2]
[x1 + 2*x2]
[x1 + 2*x2]
[x1 + 2*x2]
[x1 + 2*x2]
[x1 + 2*x2]
[x2/x3 + x4]
[x2/x3 + x4]
[x2*x3**2*x5/x4]
[x2*x3**2*x5/x4]
[x1/(x5*sin(x1*x4))]
[x4/x5]
[x4/x5]
[x4/x5]
[x4/x5]
[x4/x5]
[x4/x5]
[x4/x5]
[x2/x5 + x4]
[x2/x5 + x4]
[sin(x1/x4)]
[-x1 + x2]
[-x1 + x2]
[-x1 + x2]
[-x1 + x2]
[x2*x5]
[x2*x5]
[x2*x5]
[x2*x5]
[x4*x5]
[2*x5]
[x3 + x3/x4 + x5 + x5**2/x1]
[x3 + x3/x4 + x5 + x5**2/x1]
[x3 + x3/x4 + x5 + x5**2/x1]
[x3 + x3/x4 + x5 + x5**2/x1]
[x3 + x3/x4 + x5 + x5**2/x1]
[x3 + x3/

In [38]:
def simple_grammar(gram):
        """ Sprejme 
                gram ... gramatiko tipa grammar production
            Vrne: gramatiko transformirano v slovar {sprem_1 : {pravilo_1 : p_1, ..., pravilo_n : p_n}, sprem_2 :{}, ...}
        """
        simpl_gram = {}
        for el in gram:
            prob = el.prob()
            rull = el.rhs()
            var = el.lhs()
            try:
                simpl_gram[var][rull] = prob
            except:
                simpl_gram[var] = {rull:prob}

        return simpl_gram

In [102]:
slovar_gram = simple_grammar(grammar.grammar.productions())

#print(mm)
print(rm[0].get_error())
#print(ED.get_stats())

#print(ED.get_results(1))
#print(grammar)
hji = list(range(1,11))
#print(grammar.count_trees(Nonterminal("E"), 1))
#drevesa = [grammar.count_trees(Nonterminal("E"), h) for h in range(1, 11)]
#print(drevesa)
#print(Nonterminal("E"))
#print(grammar.code_to_expression(ED.models[0].info['code']))

code = [x for x in list(rm[0].info["trees"].keys())]

#print(grammar.code_to_expression(code[0]))

path = grammar.code_to_expression(code[0])

#for pravilo in path[1]:
slovar_path = simple_grammar(path[1])

def posodobi_gramatiko(slovar_gram, slovar_path,delta=0.01):
    n = [list(p.keys()) for p in slovar_path.values()]
    p = [item for sublist in n for item in sublist]
    for pravilo in slovar_gram.items():
        num = len(slovar_path[pravilo[0]])
        s = 0
        for pravilo_pravila in pravilo[1]:
            if pravilo_pravila in p:
                #print(pravilo_pravila)
                #print(slovar_gram[pravilo[0]][pravilo_pravila])
                slovar_gram[pravilo[0]][pravilo_pravila] += delta
                s += slovar_gram[pravilo[0]][pravilo_pravila]
                #print(slovar_gram[pravilo[0]][pravilo_pravila]) 
            elif pravilo_pravila not in p:
                #print(pravilo_pravila)
                #print(slovar_gram[pravilo[0]][pravilo_pravila])
                slovar_gram[pravilo[0]][pravilo_pravila] -= num * delta
                new = slovar_gram[pravilo[0]][pravilo_pravila]
                if new < 0:
                    slovar_gram[pravilo[0]][pravilo_pravila] = 0
                s += slovar_gram[pravilo[0]][pravilo_pravila]
        #vsota verjetnosti se ne sešteje v 1 -> normiramo
        #s = sum(pravilo[1].values())
        for pravilo_pravila in pravilo[1]:
            slovar_gram[pravilo[0]][pravilo_pravila] = slovar_gram[pravilo[0]][pravilo_pravila]/s
    return slovar_gram

new_grammar = posodobi_gramatiko(slovar_gram,slovar_path)


#print(grammar.grammar.productions())
#print(path[1][0] == grammar.grammar.productions()[0])
#s = str(grammar.grammar.productions()[0])
#print(s.find("["))
#print(s[13])

19.409958555864662


In [96]:
from nltk.grammar import ProbabilisticProduction
from nltk.grammar import PCFG

def create_grammar(simp_gram):
        """Sprejem
                simp_gram ... gramatiko v obliki slovarja
            Vrne: gramatiko tipa ProGED.GeneratorGrammmar
        """
        gram = []
        for var, dict_rull_prob in simp_gram.items():
            for rull, prob in dict_rull_prob.items():
                gram.append(ProbabilisticProduction(var, list(rull), prob=prob))
        
        return pg.GeneratorGrammar(PCFG(Nonterminal('E'), gram))

print(create_grammar(new_grammar))

Grammar with 14 productions (start state = E)
    E -> E '+' F [0.79798]
    E -> E '-' F [0.0909091]
    E -> F [0.111111]
    F -> F '*' T [0.191919]
    F -> F '/' T [0.393939]
    F -> T [0.414141]
    T -> V [0.414141]
    T -> '(' E ')' [0.292929]
    T -> 'sin' '(' E ')' [0.292929]
    V -> 'x1' [0.195876]
    V -> 'x2' [0.195876]
    V -> 'x3' [0.195876]
    V -> 'x4' [0.195876]
    V -> 'x5' [0.216495]


In [112]:
def find_eq(gramatika, delta=0.1,n_iter=10,max_diff = 0.001):
    grammar = pg.GeneratorGrammar(gramatika)
    podatki = generiraj_enacbo_1(100)
    ED = pg.EqDisco(data=podatki, 
                    sample_size=10,
                    lhs_vars=["y"],
                    rhs_vars=["x1", "x2", "x3", "x4", "x5"],
                    verbosity=1,
                    generator = grammar)
    #pg.Model
    ED.generate_models()
    mm = ED.fit_models()
    rm = ED.get_results()
    code = [x for x in list(rm[0].info["trees"].keys())]

    #print(grammar.code_to_expression(code[0]))

    path = grammar.code_to_expression(code[0])

    slovar_path = simple_grammar(path[1])
    slovar_gram = simple_grammar(grammar.grammar.productions())
    #first_error = rm[0].get_error()
    n = 0
    while n<=n_iter:
        n += 1
        dict_grammar = posodobi_gramatiko(slovar_gram,slovar_path)
        new_grammar = create_grammar(dict_grammar)
        #print(new_grammar)
        #new_grammar = pg.GeneratorGrammar(new_grammar)
        ED = pg.EqDisco(data=podatki, 
                        sample_size=10,
                        lhs_vars=["y"],
                        rhs_vars=["x1", "x2", "x3", "x4", "x5"],
                        verbosity=1,
                        generator = grammar)
        
        ED.generate_models()
        mm = ED.fit_models()
        new_best = ED.get_results()
        code = [x for x in list(new_best[0].info["trees"].keys())]

        path = grammar.code_to_expression(code[0])

        slovar_path = simple_grammar(path[1])
        slovar_gram = simple_grammar(grammar.grammar.productions())
        new_error = new_best[0].get_error()
        print(new_error)
        # ideja: če je rezultat slabši od prejšnjega -> spremeni delto!

    print(new_grammar)
    return new_best


new_best = find_eq(gramatika)
print(new_best)

[x3]
[x1 + x2]
[x2*x5]
[x1*x4 + x3]
[2*x1]
[x4 + x5]
[x4]
[x2 + x3]
[x5]
[x5]
[x5]
[x5 + x3/x1]
[x3/x1]
[x2]
[x3]
[x1]
[x1*x2]
[x1/x2]
[x2/x4 + x5]
[x2/x4 + x5]
18.105357498409287
[x2]
[x3]
[x4]
[x5/x2]
[x5/x2]
[2*x3]
[x1/(x2*x3*x4*x5)]
[x1/(x2*x3*x4*x5)]
[x1/(x2*x3*x4*x5)]
19.841170348111884
[x2*x3]
[x4 + x5/x2]
[x4]
[x1]
[x5]
[x5]
[x5]
[x1 + x4]
[x3]
[x1 + x2]
18.105357498409287
[x3/(x1*x2)]
[x2]
[x4/x2]
[x4/x2]
[x4/x2]
[x5]
[x1]
[x1 + x2]
[x1 + x2]
[x1 + x2]
18.105357498409287
[x5]
[x4]
[1/x4]
[1/x4]
[x1]
[x1]
[x1]
[x3]
[x3]
18.105357498409287
[x1*x4 + x2/(x1*x3)]
[x5]
[x5]
[x3]
[x3]
[x1*x2]
[x1*x2]
[x1]
[x2]
[x2]
18.105357498409287
[x4]
[x1]
[x1 + x2]
[x2]
[x2 + x3]
[x3]
[x3]
[x3]
[x1*x5 + x1/x4]
18.105357498409287
[x4 + x5]
[x5]
[x2]
[x2]
[x4]
[x4]
[x4]
[x4]
[x3/x4 + x5]
[x1]
18.105357498409287
[x2**2/x5 + x2*sin(x4*x5) + x4]
[x3]
[1/x2]
[x5]
[x2]
[x1/x4]
[x1*x5]
[x1]
[x1]
[1/x1]
18.105357498409287
[x5]
[x5]
[x5]
[x3]
[x3]
[x1/x2]
[x1/x2]
[x4]
[x4]
[1/(x1*x4)]
18.702259966271427
[